
# IMPLEMENTING A NEURAL NETWORK

In [61]:
from sklearn import datasets  
import numpy as np
import math 
import random 

In [30]:
# this func for computing perdictions
def prediction(model , x ) :
    w1 , w2 , b1 , b2 = model['w1'] , model['w2'] , model['b1'] , model['b2'] 
    z1 = x.dot(w1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(w2) + b2
    res = softmax(z2)
    
    return np.argmax(res, axis=1)

    

In [43]:
#this func for computing loss 
def compute_loss(model , x , y , reg_lambda):
    #get length of the data
    num_examples = len(y)
    
    #get the predictions by using this func
    w1 , w2 , b1 , b2 = model['w1'] , model['w2'] , model['b1'] , model['b2'] 
    z1 = x.dot(w1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(w2) + b2
    prob = softmax(z2)
    
    #get the loss of data 
    logprop = -np.log(prob[range(num_examples), y])
    data_loss = np.sum(logprop)
    #add the regulatization term to loss
    data_loss += reg_lambda /2  * (np.sum(np.square(w1)) + np.sum(np.square(w2)))
    
    return 1.0 / num_examples * data_loss

In [7]:
 def softmax(x):
        '''
        using softmax for the output layer
        '''
        x = np.array(x,dtype=float)

        #if the array was 1 dim
        if len(x.shape) ==1 :
            #normalize it 
            x = x - x.max()
            #calculate the softmax
            x = np.exp(x) / np.sum(np.exp(x))
        else :
            m= np.max(x, axis=1)
            #make it a coulm vector so every row get normalized by it's max
            x -= m.reshape((x.shape[0], 1))
            #calculate the softmax
            x = np.exp(x) / (np.sum(np.exp(x), axis=1)).reshape(x.shape[0], 1)

        return x

In [27]:
def gradient_descent(model , x ,y) :
    w1 , w2 , b1 , b2 = model['w1'] , model['w2'] , model['b1'] , model['b2'] 
    
    num_examples = len(x)
    
    #forward propg    
    z1 = x.dot(w1) + b1
    a1 = np.tanh(z1)
    z2 = a1.dot(w2) + b2
    exp_scores = np.exp(z2)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Backpropagation
    delta3 = probs
    delta3[range(num_examples), y] -= 1
    dW2 = (a1.T).dot(delta3)
    db2 = np.sum(delta3, axis=0, keepdims=True)
    delta2 = delta3.dot(w2.T) * (1 - np.power(a1, 2))
    dW1 = np.dot(x.T, delta2)
    db1 = np.sum(delta2, axis=0)
    #our gradients stored here 
    dpram={'dw2' : dW2 , 'dw1':dW1 , 'db1' : db1 , 'db2' : db2}
    return dpram


In [51]:
def build_model(x , y , hid_n , inp_n , out_n , reg  , eps , num_passess = 20000) :
    #inialize our arrays 
    np.random.seed(0)
    w1 = np.random.randn(inp_n , hid_n) / np.sqrt(inp_n)
    w2 = np.random.randn(hid_n , out_n) / np.sqrt(hid_n)
    b1 = np.zeros((1 , hid_n))
    b2 = np.zeros((1 , out_n))
    
    #our parameters stored here 
    model = {}
    param = {}
    # iteraion 
    model = {'w1':w1 , 'w2':w2 , 'b1' :b1 , 'b2' :b2}
    for i in range(0 , num_passess) :
        param = gradient_descent(model , x , y )
        
        #add regulariation
        dw2 = param['dw2'] + reg * w2
        dw1 = param['dw1'] + reg * w1
        db1 = param['db1']
        db2 = param['db2']
        
        #update our parameters 
        w1 -= eps * dw1
        w2 -= eps * dw2
        b1 -= eps * db1 
        b2 -= eps * db2
        
        #assign our new params 
        model = {'w1' : w1 , 'w2': w2 , 'b1' :b1 , 'b2' :b2}
        loss = compute_loss(model, x, y , reg)
        if i % 1000 == 0 :
            
            print "Loss after iteration %i: %f" % (i,loss)
        
    print "========================== /n"    
    print "print W1 " , model['w1']
    print "print W2 " , model['w2']
    print "print b1 " , model['b1']
    print "print b2 " , model['b2']
    print "print the loss"  , loss
    return model

    

# train our model 

In [52]:
eps = .01
reg = .01
hid_n = 3
inp_n = 2
out_n = 2

#generate our data
np.random.seed(0)
x, y = datasets.make_moons(200, noise=0.20)

#build our model 
model = build_model(x ,y , hid_n , inp_n , out_n , reg  , eps , 20000)


Loss after iteration 0: 0.432387
Loss after iteration 1000: 0.068947
Loss after iteration 2000: 0.068926
Loss after iteration 3000: 0.071218
Loss after iteration 4000: 0.071253
Loss after iteration 5000: 0.071278
Loss after iteration 6000: 0.071293
Loss after iteration 7000: 0.071303
Loss after iteration 8000: 0.071308
Loss after iteration 9000: 0.071312
Loss after iteration 10000: 0.071314
Loss after iteration 11000: 0.071315
Loss after iteration 12000: 0.071315
Loss after iteration 13000: 0.071316
Loss after iteration 14000: 0.071316
Loss after iteration 15000: 0.071316
Loss after iteration 16000: 0.071316
Loss after iteration 17000: 0.071316
Loss after iteration 18000: 0.071316
Loss after iteration 19000: 0.071316
========================== /n
print W1  [[ 2.93518326 -4.15545598  3.42795842]
 [ 2.6274756   0.75135518 -0.75175306]]
print W2  [[ 4.40472193 -4.34231862]
 [ 4.84301638 -4.81900141]
 [-2.66218456  2.78705795]]
print b1  [[-2.33806553  4.95656369  1.28485078]]
print b2  [[

"\nlet's train our model \n"


# change some params , and compare the results 

In [64]:

# increase the num of passes 
eps = .01
reg = .01
hid_n = 3
inp_n = 2
out_n = 2

print "the loss was .070 "
print "          "
#build our model 
model = build_model(x ,y , hid_n , inp_n , out_n , reg  , eps , 30000)



the loss was .070 
          
Loss after iteration 0: 0.432387
Loss after iteration 1000: 0.068947
Loss after iteration 2000: 0.068926
Loss after iteration 3000: 0.071218
Loss after iteration 4000: 0.071253
Loss after iteration 5000: 0.071278
Loss after iteration 6000: 0.071293
Loss after iteration 7000: 0.071303
Loss after iteration 8000: 0.071308
Loss after iteration 9000: 0.071312
Loss after iteration 10000: 0.071314
Loss after iteration 11000: 0.071315
Loss after iteration 12000: 0.071315
Loss after iteration 13000: 0.071316
Loss after iteration 14000: 0.071316
Loss after iteration 15000: 0.071316
Loss after iteration 16000: 0.071316
Loss after iteration 17000: 0.071316
Loss after iteration 18000: 0.071316
Loss after iteration 19000: 0.071316
Loss after iteration 20000: 0.071316
Loss after iteration 21000: 0.071316
Loss after iteration 22000: 0.071316
Loss after iteration 23000: 0.071316
Loss after iteration 24000: 0.071316
Loss after iteration 25000: 0.071316
Loss after iteration 

In [54]:

# increase the num of nn in hidden layer
eps = .01
reg = .01
hid_n = 4
inp_n = 2
out_n = 2

#generate our data
np.random.seed(0)
x, y = datasets.make_moons(200, noise=0.20)
print "the loss was .070 "
print "          "
#build our model 
model = build_model(x ,y , hid_n , inp_n , out_n , reg  , eps , 30000)



the loss was .070 
          
Loss after iteration 0: 0.466461
Loss after iteration 1000: 0.066685
Loss after iteration 2000: 0.060113
Loss after iteration 3000: 0.057100
Loss after iteration 4000: 0.055831
Loss after iteration 5000: 0.055171
Loss after iteration 6000: 0.054800
Loss after iteration 7000: 0.057093
Loss after iteration 8000: 0.051177
Loss after iteration 9000: 0.050966
Loss after iteration 10000: 0.050251
Loss after iteration 11000: 0.050184
Loss after iteration 12000: 0.050372
Loss after iteration 13000: 0.050205
Loss after iteration 14000: 0.050127
Loss after iteration 15000: 0.050209
Loss after iteration 16000: 0.059759
Loss after iteration 17000: 0.050977
Loss after iteration 18000: 0.050130
Loss after iteration 19000: 0.051254
Loss after iteration 20000: 0.052247
Loss after iteration 21000: 0.050127
Loss after iteration 22000: 0.052458
Loss after iteration 23000: 0.050997
Loss after iteration 24000: 0.050142
Loss after iteration 25000: 0.058594
Loss after iteration 

the loss decrease 
let's increase the nn in the hidden layer 

In [65]:
eps = .01
reg = .01
hid_n = 5
inp_n = 2
out_n = 2

print "the loss was .050 "
print "          "
#build our model 
model = build_model(x ,y , hid_n , inp_n , out_n , reg  , eps , 30000)



the loss was .050 
          
Loss after iteration 0: 0.613897
Loss after iteration 1000: 0.053880
Loss after iteration 2000: 0.044053
Loss after iteration 3000: 0.041367
Loss after iteration 4000: 0.040268
Loss after iteration 5000: 0.039732
Loss after iteration 6000: 0.039444
Loss after iteration 7000: 0.039280
Loss after iteration 8000: 0.039185
Loss after iteration 9000: 0.039128
Loss after iteration 10000: 0.039093
Loss after iteration 11000: 0.039072
Loss after iteration 12000: 0.039059
Loss after iteration 13000: 0.039050
Loss after iteration 14000: 0.039045
Loss after iteration 15000: 0.039042
Loss after iteration 16000: 0.039040
Loss after iteration 17000: 0.039038
Loss after iteration 18000: 0.039038
Loss after iteration 19000: 0.039037
Loss after iteration 20000: 0.039037
Loss after iteration 21000: 0.039036
Loss after iteration 22000: 0.039036
Loss after iteration 23000: 0.039036
Loss after iteration 24000: 0.039036
Loss after iteration 25000: 0.039036
Loss after iteration 

let's decrease the reg_lambda
 

In [56]:
eps = .01
reg = .001
hid_n = 5
inp_n = 2
out_n = 2

print "the loss was .0390359328225"
print "          "
#build our model 
model = build_model(x ,y , hid_n , inp_n , out_n , reg  , eps , 30000)



the loss was .0390359328225
          
Loss after iteration 0: 0.613664
Loss after iteration 1000: 0.047437
Loss after iteration 2000: 0.034416
Loss after iteration 3000: 0.029516
Loss after iteration 4000: 0.026723
Loss after iteration 5000: 0.024821
Loss after iteration 6000: 0.023405
Loss after iteration 7000: 0.022285
Loss after iteration 8000: 0.021368
Loss after iteration 9000: 0.021595
Loss after iteration 10000: 0.021657
Loss after iteration 11000: 0.021705
Loss after iteration 12000: 0.021729
Loss after iteration 13000: 0.021722
Loss after iteration 14000: 0.021662
Loss after iteration 15000: 0.021532
Loss after iteration 16000: 0.021338
Loss after iteration 17000: 0.021120
Loss after iteration 18000: 0.020918
Loss after iteration 19000: 0.020750
Loss after iteration 20000: 0.020615
Loss after iteration 21000: 0.020504
Loss after iteration 22000: 0.020412
Loss after iteration 23000: 0.020334
Loss after iteration 24000: 0.020267
Loss after iteration 25000: 0.020209
Loss after i

let's decrease the reg_lambda again

In [66]:
eps = .01
reg = .0001
hid_n = 5
inp_n = 2
out_n = 2

print "the loss was .0187920845245"
print "          "
#build our model 
model = build_model(x ,y , hid_n , inp_n , out_n , reg  , eps , 30000)

the loss was .0187920845245
          
Loss after iteration 0: 0.613641
Loss after iteration 1000: 0.046749
Loss after iteration 2000: 0.033328
Loss after iteration 3000: 0.028110
Loss after iteration 4000: 0.025031
Loss after iteration 5000: 0.022867
Loss after iteration 6000: 0.021200
Loss after iteration 7000: 0.019840
Loss after iteration 8000: 0.018695
Loss after iteration 9000: 0.018848
Loss after iteration 10000: 0.018484
Loss after iteration 11000: 0.018159
Loss after iteration 12000: 0.017844
Loss after iteration 13000: 0.017504
Loss after iteration 14000: 0.017122
Loss after iteration 15000: 0.016730
Loss after iteration 16000: 0.016373
Loss after iteration 17000: 0.016065
Loss after iteration 18000: 0.015799
Loss after iteration 19000: 0.015562
Loss after iteration 20000: 0.015347
Loss after iteration 21000: 0.015149
Loss after iteration 22000: 0.014965
Loss after iteration 23000: 0.014794
Loss after iteration 24000: 0.014634
Loss after iteration 25000: 0.014483
Loss after i

finally the loss become 0.0151979689841
and it would be better if we increase the num of passes again 